In [13]:
import pandas as pd
from forex_python.converter import CurrencyRates
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
# df = pd.read_csv("../Dataset/ds_salaries.csv")
c = CurrencyRates()
def calculate_actual_currency(original_value, currency):
    try:
        return original_value * c.get_rate(currency, 'USD')
    except: 
        return 0

df["calculated_salary"] = df.apply(lambda row: calculate_actual_currency(row.salary, row.salary_currency), axis=1)
df["salary_diff"] = df["salary_in_usd"] - df["calculated_salary"]
display(df.sort_values(by="salary_diff"))


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,calculated_salary,salary_diff
3594,2020,SE,FT,Data Engineer,720000,MXN,33511,MX,0,MX,S,40774.220592,-7263.220592
3469,2022,MI,FT,Data Scientist,115000,CHF,120402,CH,0,CH,L,127636.064900,-7234.064900
2379,2022,MI,FT,Machine Learning Engineer,100000,CHF,104697,CH,100,CH,L,110987.882522,-6290.882522
603,2022,SE,FT,Lead Data Scientist,164000,EUR,172309,IE,100,IE,L,177251.200000,-4942.200000
367,2023,MI,FT,Research Scientist,161200,GBP,195895,GB,0,GB,M,200627.544910,-4732.544910
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570,2021,MI,FT,Research Scientist,235000,CAD,187442,CA,100,CA,L,174262.778731,13179.221269
3725,2021,SE,FT,Principal Data Scientist,147000,EUR,173762,DE,100,DE,M,158877.600000,14884.400000
3634,2021,MI,FT,Data Engineer,250000,TRY,28016,TR,100,TR,M,12620.919337,15395.080663
3475,2021,MI,FT,ML Engineer,8500000,JPY,77364,JP,50,JP,S,61429.622200,15934.377800


In [2]:
df_features = df.copy(deep=True)
df_features_copy = df_features.copy(deep=True)
df_features.drop_duplicates(inplace=True)
target = df_features["salary_in_usd"]
df_features.drop(["salary", "salary_in_usd"], axis=1, inplace=True)


display(df_features)

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,EUR,ES,100,ES,L
1,2023,MI,CT,ML Engineer,USD,US,100,US,S
2,2023,MI,CT,ML Engineer,USD,US,100,US,S
3,2023,SE,FT,Data Scientist,USD,CA,100,CA,M
4,2023,SE,FT,Data Scientist,USD,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,USD,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,USD,US,100,US,L
3752,2020,EN,FT,Data Scientist,USD,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,USD,US,100,US,L


# Create preprocessor

In [3]:

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
import pycountry_convert as pc

def get_continent(country):
    country_continent_code = pc.country_alpha2_to_continent_code(country)
    return country_continent_code

df_features["residence"] = df_features["employee_residence"].apply(get_continent)

oneHot_features = ["work_year", "job_title", "residence", "salary_currency", "experience_level", "company_size"]
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), oneHot_features),
    ])

oneHot_features_copy = ["work_year", 'salary_currency', 'employee_residence', "company_location", "employment_type", "remote_ratio", "job_title", "company_size", "experience_level"]
preprocessor_copy = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), oneHot_features_copy),
    ])

df_preprocessed = preprocessor.fit_transform(df_features)

df_preprocessed_copy = preprocessor_copy.fit_transform(df_features_copy)

df_preprocessed_features = pd.DataFrame(data=df_preprocessed.toarray(), columns=preprocessor.get_feature_names_out())
df_preprocessed_features_copy = pd.DataFrame(data=df_preprocessed_copy.toarray(), columns=preprocessor_copy.get_feature_names_out())
print(len(df_preprocessed_features_copy.columns))
print(len(df_preprocessed_features.columns))

281
130


In [4]:
from sklearn.model_selection import train_test_split

target_stratified = pd.cut(target, bins=6, labels=False)

print(target_stratified)

0       1
1       0
2       0
3       2
4       1
       ..
3750    5
3751    1
3752    1
3753    1
3754    1
Name: salary_in_usd, Length: 2584, dtype: int64


In [5]:
# Split into 80/20 Training / Testing
X_train, X_holdout, target_train, target_holdout = train_test_split(df_preprocessed, target, test_size=0.2, random_state=42)

X_train_stratified, X_holdout_stratified, target_train_stratified, target_holdout_stratified = train_test_split(df_preprocessed, target, test_size=0.2, stratify=target_stratified, random_state=42)

In [6]:
from scipy.sparse import save_npz

save_npz('./ds_salaries_Residence_Clustering_train.npz', X_train)
save_npz('./ds_salaries_Residence_Clustering_train_stratified.npz', X_train_stratified)

target_train.to_csv('./ds_salaries_target_train.csv', index=False)
target_train_stratified.to_csv('./ds_salaries_target_train_stratified.csv', index=False)

save_npz('./ds_salaries_Residence_Clustering_test.npz', X_holdout)
save_npz('./ds_salaries_Residence_Clustering_test_stratified.npz', X_holdout_stratified)

target_holdout_stratified.to_csv('./ds_salaries_target_test_stratified.csv', index=False)
target_holdout.to_csv('./ds_salaries_target_test.csv', index=False)

display(target_train.describe())
display(target_holdout.describe())


count      2067.000000
mean     133570.320271
std       67383.813058
min        5132.000000
25%       85000.000000
50%      130000.000000
75%      175000.000000
max      450000.000000
Name: salary_in_usd, dtype: float64

count       517.000000
mean     132765.431335
std       66200.821478
min        5409.000000
25%       80000.000000
50%      130000.000000
75%      180000.000000
max      430967.000000
Name: salary_in_usd, dtype: float64